step 1:

In [2]:
### run this step in jupyter.

%cd ~/522/test
  
!git clone --quiet https://github.com/tensorflow/models.git

# !sudo -S apt-get install -qq protobuf-compiler python-tk

import getpass
import os

password = getpass.getpass()
command = "sudo -S apt-get install -qq protobuf-compiler python-tk"
os.system('echo %s | %s' % (password, command))

!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive

!pip install -q pycocotools

%cd ~/522/test/models/research
!protoc object_detection/protos/*.proto --python_out=.

step 2:

In [ ]:
### run this step in pycharm. Configure the sources root and PYTHONPATH in pycharm, then run model_builder_test.py

# import os
# os.environ['PYTHONPATH'] += ':/home/justin/522/test/models/research/:/root/522/test/models/research/slim/'

!python object_detection/builders/model_builder_test.py

step 3:

In [1]:
### this step can be achieve manually or run by jupyter(but path and code need to be modified)

%cd ~/522/test
%mkdir datalab
%cd datalab

!echo "item {\n id: 1\n name: 'dog'\n}" > label_map.pbtxt

fileId = ''

### this part download the dog-dataset.zip and then unzip(folder structure is modified)
import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fileName = fileId + '.zip'
downloaded = drive.CreateFile({'id': fileId})
downloaded.GetContentFile(fileName)
ds = ZipFile(fileName)
ds.extractall()
os.remove(fileName)
print('Extracted zip file ' + fileName)

### this part generate trainval.txt file
image_files=os.listdir('images')
im_files=[x.split('.')[0] for x in image_files]
with open('annotations/trainval.txt', 'w') as text_file:
  for row in im_files:
    text_file.write(row + '\n')

In [1]:
# import os

# %cd ~/522/test/datalab

# image_files=os.listdir('images')
# im_files=[x.split('.')[0] for x in image_files]
# with open('annotations/trainval.txt', 'w') as text_file:
#   for row in im_files:
#     text_file.write(row + '\n')

/home/justin/522/test/datalab


step 4: 

In [2]:
### this step can be achieve manually or run by jupyter(but paths and code need to be modified)
### it generates blank image under folder trimaps for each file in xmls

%cd ~/522/test/datalab/annotations

!mkdir trimaps

from PIL import Image
image = Image.new('RGB', (640, 480))

for filename in os.listdir('xmls'):
  filename = os.path.splitext(filename)[0]
  image.save('trimaps/' + filename + '.png')

/home/justin/522/test/datalab/annotations


step 5:

In [ ]:
### run this step in pycharm with parameter configured, then rename those two files

%cd ~/datalab

!python ~/models/research/object_detection/dataset_tools/create_pet_tf_record.py --label_map_path=label_map.pbtxt --data_dir=. --output_dir=. --num_shards=1

!mv pet_faces_train.record-00000-of-00001 tf_train.record

!mv pet_faces_val.record-00000-of-00001 tf_val.record

step 6:

In [1]:
### run this step in jupyter, cd path needs to be modified

%cd ~/522/test/datalab

import os
import shutil
import glob
import urllib
import tarfile

MODEL = 'faster_rcnn_inception_v2_coco_2018_01_28'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = 'pretrained_model'

if not (os.path.exists(MODEL_FILE)):
  opener = urllib.URLopener()
  opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
  shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/home/justin/522/test/datalab


step 7:

In [3]:
### run this step in jupyter, cd path and PATH_TO_BE_CONFIGURED paths needs to be modified to local ones

%cd ~/522/test/datalab

import re

#filename = '/content/datalab/pretrained_model/pipeline.config'
filename = '/home/justin/522/test/models/research/object_detection/samples/configs/faster_rcnn_inception_v2_pets.config'
with open(filename) as f:
  s = f.read()
with open(filename, 'w') as f:
  s = re.sub('PATH_TO_BE_CONFIGURED/model.ckpt', '/home/justin/522/test/datalab/pretrained_model/model.ckpt', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/pet_faces_train.record-\?\?\?\?\?-of-00010', '/home/justin/522/test/datalab/tf_train.record', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/pet_faces_val.record-\?\?\?\?\?-of-00010', '/home/justin/522/test/datalab/tf_val.record', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/pet_label_map.pbtxt', '/home/justin/522/test/datalab/label_map.pbtxt', s)
  f.write(s)

/home/justin/522/test/datalab


step 8:

In [4]:
### run this step in pycharm, with parameter configered
### also, can use Tensorboard during the process to get some charts

# %cd ~/522/test/datalab

!python /home/justin/522/test/models/research/object_detection/model_main.py \
    --pipeline_config_path=/home/justin/522/test/models/research/object_detection/samples/configs/faster_rcnn_inception_v2_pets.config \
    --model_dir=/home/justin/522/test/datalab/trained \
    --alsologtostderr \
    --num_train_steps=3000 \
    --num_eval_steps=500

step 9:

In [1]:
### run this step in jupyter to get the value of last_model.

%cd ~/522/test/datalab

import os

lst = os.listdir('trained')
lf = filter(lambda k: 'model.ckpt-' in k, lst)
last_model = sorted(lf)[-1].replace('.meta', '')
print last_model

/home/justin/522/test/datalab
model.ckpt-3000


data example in this step: 
lst: all the files in directory trained (['model.ckpt-2494.data-00000-of-00001', 'graph.pbtxt', 'model.ckpt-2905.index',...])
lf: files that contains 'model.ckpt-' (['model.ckpt-2494.data-00000-of-00001', 'model.ckpt-2905.index',...])
last_model: the file with largest checkpoint number (model.ckpt-3322)

step 10:

In [ ]:
### run this step in pycharm, with parameters set according to the result of the previous step
### $last_model means the value of variable last_model

!python ~/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/home/justin/522/test/models/research/object_detection/samples/configs/faster_rcnn_inception_v2_pets.config \
    --output_directory=/home/justin/522/test/datalab/fine_tuned_model \
    --trained_checkpoint_prefix=/home/justin/522/test/datalab/trained/$last_model

step 11:

In [3]:
### run this step in jupyter. the image_path is the file path of the image for which u want to run object dectection

%cd /home/justin/522/test/datalab

image_path = '/home/justin/522/blimp_0068.jpg'

with open (image_path) as f:
    image = f.read()
with open ('image1.jpg', 'wb') as f:
    f.write(image)
    f.close()
    print('saved file')

/home/justin/522/test/datalab
saved file


step 12:

In [ ]:
### run this step in pycharm. new a python file in model and copy the following code into it.(name it run_inference.py)
### then run it. 

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# # This is needed since the notebook is stored in the object_detection folder.
# sys.path.append("..")
from object_detection.utils import ops as utils_ops

# if tf.__version__ < '1.4.0':
#     raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')

# # This is needed to display the images.
# % matplotlib inline

from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

# What model to download.
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '/home/justin/522/test/datalab/fine_tuned_model' + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('/home/justin/522/test/datalab', 'label_map.pbtxt')

NUM_CLASSES = 37

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES,
                                                            use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)


# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = '/home/justin/522/test/datalab/'
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 2)]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict


for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)
    plt.show()